# Homework 1

## 1.1

This topic of introspection is difficult to apply in AI especially since there is no set science of how it is done with humans. Introspection is where humans examine their own minds without outside influence. On the one hand, humans are shaped by outside influences from the time that they were born, so the thoughts they have at any point in their life, even simply within their own minds, are still influenced by their experience with the outside world. This can be seen the same way inside of machines – everything they “know” has been influenced by the outside world (humans programming them, the internet, etc.).

On the other hand, the definition of introspection includes two words that are often discounted when it comes to machines – consciousness and feelings. It is the examination of one’s own conscious thoughts and feelings. Humans are conscious, but machines are not. Humans can have feelings, but machines cannot. Consequently, is it possible for machines to practice introspection if they do not even possess the very principles it is based upon? I do not think this is possible, however, I do think that programming machines to learn from their “experiences in this world” is quite possible and prove to be much more efficient. I would call this process machine learning, though, rather than introspection.


## 1.2

Problem Formulation:

State representation:
    [c1, c2, ..., cn] gives the order in which the cities are visited.
Move/action representation:
    [num1, num2]: Swap the positions (within the list) of the cities at the given indexes.

The sample city-domain consists of five cities labeled a, b, c, d, and e. Simulated annealing does better than hill-climbing most of the time (sometimes, they come up with the same answers). Simulated annealing works better because it is willing to try an action that is "bad" for the sake of finding a better path afterwards. However, hill-climbing will only choose the best option in front of it at the moment and will never choose a "bad" city to go to even if that one is better in the end.

In [ ]:
"""
This implements a local-search version of a TSP problem for AIMA-Python.
Based on kvlinden's queens.py.

Solution: Simulated annealing does better than hill-climbing most of the time
(sometimes, they come up with the same answers). Simulated annealing works
better because it is willing to try an action that is "bad" for the sake of
finding a better path afterwards. However, hill-climbing will only choose the
best option in front of it at the moment and will never choose a "bad" city to
go to even if that one is better in the end.

@authors: Sarah Griffioen (slg27) & Lydia Holtrop (ljh27)
"""
from tools.aima.search import Problem, hill_climbing, simulated_annealing, exp_schedule
from random import randrange


class TSP(Problem):
    """An implementation of a TSP problem for local search.

    State representation: 
        [c1, c2, ..., cn] gives the order in which the cities are visited.
    Move/action representation:
        [num1, num2]: Swap the positions (within the list) of the cities at the given indexes.
    """

    def __init__(self, distances, initial):
        self.distances = distances
        self.initial = initial

    def actions(self, state):
        """This method generates two random integers to be used as indexes in a move.
        """
        num_cities = len(state) - 1 #minus 1 because start and end in same city
        actions = []
        for i in range(6):
            num1 = randrange(1, num_cities)
            num2 = randrange(1, num_cities)
            action = [num1, num2]
            actions.append(action)
        return actions

    def result(self, state, move):
        """Makes the given move (swaps the positions of the cities at the given
        indexes) on a copy of the given state."""
        new_state = state[:]
        new_state[move[0]], new_state[move[1]] = new_state[move[1]], new_state[move[0]]
        return new_state

    def value(self, state):
        """This method computes the total distance traveled in the given state by adding
        up the distance of each segment, then multiplying it by -1 to more semantically
        reflect that greater distance is less ideal.
        """
        num_visits = len(state)
        value = 0

        for i in range(1, num_visits):
            try:
                value += distances[(state[i-1], state[i])]
            except KeyError:
                value += distances[(state[i], state[i - 1])]

        value = -1 * value

        return value


if __name__ == '__main__':

    # Initialize a path and a dictionary of distances between cities
    initial_path = ['a', 'b', 'c', 'd', 'e', 'a']
    distances = {('a', 'b'): 1, ('a', 'c'): 3, ('a', 'd'): 2, ('a', 'e'): 4, ('b', 'c'): 3, ('b', 'd'): 2, ('b', 'e'): 1, ('c', 'd'): 8, ('c', 'e'): 3, ('d', 'e'): 5}

    # Initialize the TSP problem
    p = TSP(distances, initial_path)

    # Solve the problem using hill climbing.
    hill_solution = hill_climbing(p)
    print('Hill-climbing:')
    print('\tSolution: ' + str(hill_solution))
    print('\tDistance:    ' + str(-p.value(hill_solution)))

    # Solve the problem using simulated annealing.
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=10000)
    )
    print('Simulated annealing:')
    print('\tSolution: ' + str(annealing_solution))
    print('\tDistance:    ' + str(-p.value(annealing_solution)))

    if (p.value(hill_solution) > p.value(annealing_solution)):
        print("\nHill-climbing did better than simulated annealing.")
    elif (p.value(annealing_solution) > p.value(hill_solution)):
        print("\nSimulated annealing did better than hill-climbing.")
    else:
        print("\nThe two algorithms came up with the same solution.")

## 1.3

This implements a course scheduling domain using the AIMA constraint satisfaction framework.
I solved this problem based on the logic from the zebra problem. Both define variables, which,
in this case, are the classes. Then I set the domain for each of the classes, which is every
combination of professors, times, and classrooms a class could have. Lastly, I created a
dictionary of every possible class that could be taught at the same time as another class to
create the neighbors. After all of that was finished, I used min_conflicts to solve the problem.
This was all based on the general setup of the zebra problem that is laid out in csp.py.

In [1]:
"""
This implements a course scheduling domain using the AIMA constraint satisfaction framework.
I solved this problem based on the logic from the zebra problem. Both define variables, which,
in this case, are the classes. Then I set the domain for each of the classes, which is every
combination of professors, times, and classrooms a class could have. Lastly, I created a
dictionary of every possible class that could be taught at the same time as another class to
create the neighbors. After all of that was finished, I used min_conflicts to solve the problem.
This was all based on the general setup of the zebra problem that is laid out in csp.py.

@author: Sarah Griffioen (slg27)
@date: 2/23/2019
"""

from tools.aima.csp import CSP, min_conflicts

#finds all possible pairs of times and rooms
def time_room_pairs(times, rooms):
    timeRoomList = []
    for i in range(len(times)):
        for j in range(len(rooms)):
            timeRoomList.append(times[i] + ',' + rooms[j])
    return timeRoomList

#finds all possibilities of profs, times, and rooms for each class
def course_assignment_pairs(assignments, timeRoomList):
    courseAssignmentDict = {}
    for key, value in assignments.items():
        permutations = []
        for i in range(len(timeRoomList)):
            permutations.append(value + ',' + timeRoomList[i])
        courseAssignmentDict[key] = permutations
    return courseAssignmentDict

#finds all possible course pairs
def create_neighbors(courses):
    neighborDict = {}
    for i in range(len(courses)):
        validCoursesList = []
        for j in range(len(courses)):
            if(courses[j] != courses[i]):
                validCoursesList.append(courses[j])
        neighborDict[courses[i]] = validCoursesList
    return neighborDict

#apply constraints
def faculty_constraint(course1, slot1, course2, slot2):
    detailsList1 = slot1.split(',')
    detailsList2 = slot2.split(',')

    #assign variables to time, room, and faculty
    prof1 = detailsList1[0]
    prof2 = detailsList2[0]
    time1 = detailsList1[1]
    time2 = detailsList2[1]
    place1 = detailsList1[2]
    place2 = detailsList2[2]

    #return true only if there are no time/faculty or time/place conflicts
    return ((time1 != time2) or (prof1 != prof2) and (time1 != time2) or (place1 != place2))

#define variables and set domains and neighbors
courses = ['cs108', 'cs112', 'cs212', 'cs214', 'cs232', 'cs262', 'cs342']
assignments = {'cs108': 'schuurman', 'cs112': 'adams', 'cs212': 'plantinga', 'cs214': 'schuurman', 'cs232': 'adams', 'cs262': 'bailey', 'cs342': 'bailey'}
classtimes = ['mwf0900', 'mwf1030', 'mwf1130', 'mwf1230']
classrooms = ['nh253', 'sb382']
domains = course_assignment_pairs(assignments, time_room_pairs(classtimes, classrooms))
neighbors = create_neighbors(courses)

#calculate and print out results
print('\nVariables: ' + '\n' + str(courses))
print('Domains: ' + '\n' + str(domains))
print('Neighbors: ' + '\n' + str(neighbors))
solution = min_conflicts(CSP(courses, domains, neighbors, faculty_constraint))
if solution is None:
    print('no solution')
else:
    print('\nSolution:\n\nClass\tProfessor\tTime\t\tClassroom\n-----------------------------------------')
    for key, value in solution.items():
        detailsList = value.split(',')
        if (detailsList[0] == 'adams' or detailsList[0] == 'bailey'):
            detailsList[0] += '\t'
        print(key + '\t' + detailsList[0] + '\t' + detailsList[1] + '\t\t' + detailsList[2])


Here's yet more code
